Load the data.

In [3]:
import csv
import time
import random
import math
import statistics
import numpy as np
import matplotlib as mpl
import scipy.stats as sps

from datetime import datetime as dt

survey_files=["./survey_fcasts.yr1.csv", "./survey_fcasts.yr2.csv", "./survey_fcasts.yr3.csv"]
#, "./survey_fcasts.yr4.csv"]

MIN_PROB=0.01
EXTR=3

In [5]:
# List of individual forecasts. Type: List of dictionaries.
# Fields:
# {'ifp_id', 'ctt', 'cond', 'training', 'team', 'user_id', 'forecast_id', 'fcast_type', 'answer_option', 'value', 'fcast_date', 'expertise', 'q_status', 'viewtime', 'year', 'timestamp'}

preds=[]


for n in survey_files:
    f=open(n)
    forecast_reader=csv.DictReader(f)
    for entry in forecast_reader:
        entry['cond']=int(entry['cond'])
        entry['value']=float(entry['value'])
        entry['timestamp']=dt.fromisoformat(entry['timestamp'])
        entry['year']=entry['timestamp'].year
        preds.append(entry)

# List of individual questions. Type: List of dictionaries.
# Fields:
# {'ifp_id', 'q_type', 'q_text', 'q_desc', 'q_status', 'date_start', 'date_suspend', 'date_to_close', 'date_closed', 'outcome', 'short_title', 'days_open', 'n_opts', 'options'}

qdata=[]

qfile=open("ifps.csv")
qreader=csv.DictReader(qfile)

for entry in qreader:
    if entry['date_start']!='NULL':
        entry['date_start']=dt.strptime(entry['date_start'], '%m/%d/%y')
    if entry['date_suspend']!='NULL':
        entry['date_suspend']=dt.strptime(entry['date_suspend'], '%m/%d/%y %H:%M')
    qdata.append(entry)

Functions.

Problem with the code below: The Brier score has two different formulations (one equivalent to the mean squared error, another one being the original formulation by Brier). Only the original formulation is proper for forecasts on >2 options.

TODO: implement the original formulation: $BS=\frac{1}{N} \sum_{t=1}^{N} \sum_{i=1}^R (f_{ti}-o_{ti})^2$

In [6]:
def group_score(user_forecasts, users):
    forecasts=np.array([])
    results=np.array([])
    for u in users:
        udata=user_forecasts[u]
        forecasts=np.append(forecasts, np.array(udata['forecasts']))
        results=np.append(results, np.array(udata['options'])==np.array(udata['outcomes']))
    return np.mean((forecasts-results)**2)

def individual_score(user):
    return np.mean((np.array(user['forecasts'])-(np.array(user['options'])==np.array(user['outcomes'])))**2)

Different structures of the data.

In [7]:
# Outcomes of questions.
# Type: Dictionary.
# Fields: Keys are question ids (`ifp_id`), values are dictionaries (fields are `year`, `outcome`, `date_start`, `date_suspend`)

questions=dict()

for q in qdata:
    qid=q['ifp_id']
    if not qid in questions:
        questions[qid]=dict()
    questions[qid]['year']=q['date_start'].year
    questions[qid]['outcome']=q['outcome']
    questions[qid]['date_start']=q['date_start']
    questions[qid]['date_suspend']=q['date_suspend']
    questions[qid]['q_status']=q['q_status']

for p in preds:
    p['outcome']=questions[p['ifp_id']]['outcome']

In [8]:
# All forecasts, per user.
# Type: Dictionary of dictionaries. ("Look on my works, ye Mighty, and despair!")
# Fields: Keys are user ids (`user_id`), values are dictionaries, where:
# keys are 'forecasts', 'options' and 'outcomes'.

user_forecasts=dict()

for p in preds:
    uid=p['user_id']
    if not user_forecasts.__contains__(uid):
        user_forecasts[uid]=dict()
        user_forecasts[uid]['forecasts']=[]
        user_forecasts[uid]['options']=[]
        user_forecasts[uid]['outcomes']=[]
    user_forecasts[uid]['forecasts'].append(p['value'])
    user_forecasts[uid]['options'].append(p['answer_option'])
    user_forecasts[uid]['outcomes'].append(p['outcome'])

In [9]:
# In the file survey_fcasts.yr1.csv there are a couple of forecasts made by the user "NULL"
# I don't know what this means: I suspected at first that those were forecasts on "voided" questions,
# but it turns out that that's not the case.
# To be sure, I'll delete the forecasts by that user from the data.

if 'NULL' in user_forecasts.keys():
    user_forecasts.pop('NULL')
users=list(user_forecasts.keys())

In [10]:
# Sorted list of all scores individuals have received.
# Type: List of floats.

individual_scores=np.array([])

for k in user_forecasts.keys():
    u=user_forecasts[k]
    u['brier']=individual_score(u)
    individual_scores=np.append(individual_scores, u['brier'])

individual_scores=np.sort(individual_scores)

In [11]:
# Which teams to associate to each forecaster.
# Type: Dictionary of dictionaries of sets.
# Fields: Keys are user ids (`user_id`). Values
# are dictionaries, where the keys are the years,
# and the values are sets of team ids, usually
# (but not always!) unique.

user_teams=dict()

for p in preds:
    uid=p['user_id']
    if not uid in user_teams:
        user_teams[uid]=dict()
    if not p['year'] in user_teams[uid]:
        user_teams[uid][p['year']]=set() 
    if not p['team'] in user_teams[uid][p['year']]:
        user_teams[uid][p['year']].add(p['team'])

for k in user_teams:
    for y in user_teams[k]:
        if len(user_teams[k][y])!=1:
            print(k, y, user_teams[k][y])

5187 2012 {'1', 'NA'}
1456 2013 {'1', 'NA'}
4999 2013 {'1', 'NA'}
1533 2013 {'2', 'NA'}
3126 2013 {'1', 'NA'}
634 2013 {'1', 'NA'}
4980 2013 {'2', 'NA'}
4647 2013 {'2', 'NA'}
5182 2013 {'8', 'NA'}
3494 2013 {'1', 'NA'}
2938 2013 {'1', 'NA'}
2961 2012 {'2', 'NA'}
5147 2013 {'2', 'NA'}
770 2013 {'2', 'NA'}
2803 2013 {'1', 'NA'}
2200 2013 {'1', 'NA'}
3909 2013 {'2', 'NA'}
2121 2013 {'2', 'NA'}
1658 2013 {'1', 'NA'}
506 2013 {'1', 'NA'}
2592 2013 {'2', 'NA'}
3481 2013 {'2', 'NA'}
1120 2013 {'1', 'NA'}
4981 2013 {'1', '21'}
4510 2012 {'7', 'NA'}
4510 2013 {'7', '10'}
685 2012 {'NA', '12'}
121 2012 {'1', 'NA'}
121 2013 {'1', '6'}
866 2013 {'1', 'NA'}
228 2013 {'2', 'NA'}
2382 2012 {'1', 'NA'}
51 2013 {'2', 'NA'}
3197 2012 {'5', 'NA'}
3197 2013 {'38', '5'}
3946 2012 {'2', 'NA'}
5052 2012 {'16', 'NA'}
5052 2013 {'29', '16'}
2880 2012 {'16', 'NA'}
2880 2013 {'8', '16'}
3895 2012 {'10', 'NA'}
3708 2012 {'1', '2'}
4282 2013 {'1', 'NA'}
4212 2012 {'NA', '6'}
1526 2012 {'7', 'NA'}
1526 2013 {'7', '

In [12]:
# For each question, which forecaster made which prediction when?
# Type: dictionary of dictionaries of dictionaries (!) of lists
# Fields: Keys are question ids (`ifp_id`), values are dictionaries
# where keys are `year`, `forecasters`, `outcome`, `date_start` and
# `date_suspend`; `forecasters` is a dictionary where the keys are
# user ids (`user_id`) and values are dictionaries, where keys are
# `forecasts`, `options`, `timestamps` (all numpy lists).

q_fsters=dict()

# collect the data

for p in preds:
    qid=p['ifp_id']
    if not qid in q_fsters:
        q_fsters[qid]=dict()
        q_fsters[qid]['year']=questions[qid]['year']
        q_fsters[qid]['outcome']=questions[qid]['outcome']
        q_fsters[qid]['date_start']=questions[qid]['date_start']
        q_fsters[qid]['date_suspend']=questions[qid]['date_suspend']
        q_fsters[qid]['q_status']=questions[qid]['q_status']
        q_fsters[qid]['forecasters']=dict()
    uid=p['user_id']
    if not uid in q_fsters[qid]['forecasters']:
        q_fsters[qid]['forecasters'][uid]=dict()
        q_fsters[qid]['forecasters'][uid]['forecasts']=[]
        q_fsters[qid]['forecasters'][uid]['options']=[]
        q_fsters[qid]['forecasters'][uid]['timestamps']=[]
    q_fsters[qid]['forecasters'][uid]['forecasts'].append(p['value'])
    q_fsters[qid]['forecasters'][uid]['options'].append(p['answer_option'])
    q_fsters[qid]['forecasters'][uid]['timestamps'].append(p['timestamp'])

# sort the data

for qid in q_fsters.keys():
    for uid in q_fsters[qid]['forecasters'].keys():
        q_fsters[qid]['forecasters'][uid]['timestamps']=np.array(q_fsters[qid]['forecasters'][uid]['timestamps'])
        indices=np.argsort(q_fsters[qid]['forecasters'][uid]['timestamps'])
        q_fsters[qid]['forecasters'][uid]['timestamps']=q_fsters[qid]['forecasters'][uid]['timestamps'][indices]
        q_fsters[qid]['forecasters'][uid]['forecasts']=np.array(q_fsters[qid]['forecasters'][uid]['forecasts'])[indices]
        q_fsters[qid]['forecasters'][uid]['options']=np.array(q_fsters[qid]['forecasters'][uid]['options'])[indices]

        # replacing forecasts with probability 0, 1 with forecasts with probability 0.01, 0.99
        q_fsters[qid]['forecasters'][uid]['forecasts'][np.where(q_fsters[qid]['forecasters'][uid]['forecasts']==0)]=MIN_PROB
        q_fsters[qid]['forecasters'][uid]['forecasts'][np.where(q_fsters[qid]['forecasters'][uid]['forecasts']==1)]=1-MIN_PROB

In [13]:
# Forecasts made by (forecasters in?) teams, and forecasters outside of teams

team_forecasts=dict()
nonteam_forecasts=dict()

for qid in q_fsters.keys():
    year=q_fsters[qid]['year']
    team_forecasts[qid]=dict()
    nonteam_forecasts[qid]=dict()
    for uid in q_fsters[qid]['forecasters'].keys():
        for i in range(0, len(q_fsters[qid]['forecasters'][uid]['options'])-1):
            if not year in user_teams[uid]:
                continue
            opt=q_fsters[qid]['forecasters'][uid]['options'][i]
            if not opt in nonteam_forecasts[qid]:
                nonteam_forecasts[qid][opt]=dict()
                nonteam_forecasts[qid][opt]['forecasts']=[]
                nonteam_forecasts[qid][opt]['timestamps']=[]
            if not opt in team_forecasts[qid]:
                team_forecasts[qid][opt]=dict()
                team_forecasts[qid][opt]['forecasts']=[]
                team_forecasts[qid][opt]['timestamps']=[]
            if 'NA' in user_teams[uid][year]:
                nonteam_forecasts[qid][opt]['forecasts'].append(q_fsters[qid]['forecasters'][uid]['forecasts'][i])
                nonteam_forecasts[qid][opt]['timestamps'].append(q_fsters[qid]['forecasters'][uid]['timestamps'][i])
            else:
                team_forecasts[qid][opt]['forecasts'].append(q_fsters[qid]['forecasters'][uid]['forecasts'][i])
                team_forecasts[qid][opt]['timestamps'].append(q_fsters[qid]['forecasters'][uid]['timestamps'][i])

for qid in team_forecasts.keys():
    for o in team_forecasts[qid].keys():
        team_forecasts[qid][o]['forecasts']=np.array(team_forecasts[qid][o]['forecasts'])
        team_forecasts[qid][o]['timestamps']=np.array(team_forecasts[qid][o]['timestamps'])
        nonteam_forecasts[qid][o]['forecasts']=np.array(nonteam_forecasts[qid][o]['forecasts'])
        nonteam_forecasts[qid][o]['timestamps']=np.array(nonteam_forecasts[qid][o]['timestamps'])

In [14]:
# Forecasts per question

q_forecasts=dict()

for qid in q_fsters.keys():
    q_forecasts[qid]=dict()
    for uid in q_fsters[qid]['forecasters'].keys():
        for i in range(0, len(q_fsters[qid]['forecasters'][uid]['options'])-1):
            opt=q_fsters[qid]['forecasters'][uid]['options'][i]
            if not opt in q_forecasts[qid]:
                q_forecasts[qid][opt]=dict()
                q_forecasts[qid][opt]['forecasts']=[]
                q_forecasts[qid][opt]['timestamps']=[]
            q_forecasts[qid][opt]['forecasts'].append(q_fsters[qid]['forecasters'][uid]['forecasts'][i])
            q_forecasts[qid][opt]['timestamps'].append(q_fsters[qid]['forecasters'][uid]['timestamps'][i])

for qid in q_forecasts.keys():
    for o in q_forecasts[qid].keys():
        q_forecasts[qid][o]['forecasts']=np.array(q_forecasts[qid][o]['forecasts'])
        q_forecasts[qid][o]['timestamps']=np.array(q_forecasts[qid][o]['timestamps'])

In [15]:
# aggregations methods

def generate_aggr_methods():
    # different per-question aggregation methods
    # {arithmetic mean, geometric mean, median}×{probs, odds, log odds}×{exponential decay, no decay}×{extremized, not extremized}
    means=['arith', 'geom', 'median']
    formats=['probs', 'odds', 'logodds']
    decay=['nodec', 'dec']
    extremize=['gjpextr', 'postextr', 'neyextr', 'befextr', 'noextr']

    aggr_methods=[]

    for i1 in means:
        for i2 in formats:
            for i3 in decay:
                for i4 in extremize:
                    #the first sometimes doesn't work (negative values), the second is equivalent to the geometric mean of odds
                    if (i1=='geom' and i2=='logodds') or (i1=='arith' and i2=='logodds') or (i3=='dec' and i1!='arith'):
                        continue
                    aggr_methods.append([i1, i2, i3, i4])

    return aggr_methods

print(generate_aggr_methods())

[['arith', 'probs', 'nodec', 'gjpextr'], ['arith', 'probs', 'nodec', 'postextr'], ['arith', 'probs', 'nodec', 'neyextr'], ['arith', 'probs', 'nodec', 'befextr'], ['arith', 'probs', 'nodec', 'noextr'], ['arith', 'probs', 'dec', 'gjpextr'], ['arith', 'probs', 'dec', 'postextr'], ['arith', 'probs', 'dec', 'neyextr'], ['arith', 'probs', 'dec', 'befextr'], ['arith', 'probs', 'dec', 'noextr'], ['arith', 'odds', 'nodec', 'gjpextr'], ['arith', 'odds', 'nodec', 'postextr'], ['arith', 'odds', 'nodec', 'neyextr'], ['arith', 'odds', 'nodec', 'befextr'], ['arith', 'odds', 'nodec', 'noextr'], ['arith', 'odds', 'dec', 'gjpextr'], ['arith', 'odds', 'dec', 'postextr'], ['arith', 'odds', 'dec', 'neyextr'], ['arith', 'odds', 'dec', 'befextr'], ['arith', 'odds', 'dec', 'noextr'], ['geom', 'probs', 'nodec', 'gjpextr'], ['geom', 'probs', 'nodec', 'postextr'], ['geom', 'probs', 'nodec', 'neyextr'], ['geom', 'probs', 'nodec', 'befextr'], ['geom', 'probs', 'nodec', 'noextr'], ['geom', 'odds', 'nodec', 'gjpextr

In [16]:
# differently aggregated means

def get_aggregations(q_forecasts, questions):
    aggregations=dict()

    aggr_methods=generate_aggr_methods()

    for a in aggr_methods:
        aggrkey='_'.join(a)
        aggregations[aggrkey]=dict()
        for qid in q_forecasts.keys():
            aggregations[aggrkey][qid]=dict()
            aggregations[aggrkey][qid]['outcome']=questions[qid]['outcome']
            aggregations[aggrkey][qid]['aggr_forecasts']=[]
            aggregations[aggrkey][qid]['options']=[]

            for o in q_forecasts[qid].keys():
                n=len(q_forecasts[qid][o]['forecasts'])
                poss_transformed=q_forecasts[qid][o]['forecasts']

                if len(poss_transformed)==0:
                    continue

                if 'befextr' in a:
                    p=poss_transformed
                    poss_transformed=((p**EXTR)/(((p**EXTR)+(1-p))**(1/EXTR)))

                if 'dec' in a:
                    if q_fsters[qid]['date_suspend']=='NULL': #ARGH
                        weights=np.ones_like(poss_transformed)
                        break
                    t_diffs=q_fsters[qid]['date_suspend']-q_forecasts[qid][o]['timestamps']
                    t_diffs=np.array([t.total_seconds() for t in t_diffs])
                    weights=0.99**(1/(1*86400)*t_diffs)
                else:
                    weights=np.ones_like(poss_transformed)

                if 'odds' in a:
                    poss_transformed=poss_transformed/(1-poss_transformed)
                elif 'logodds' in a:
                    poss_transformed=poss_transformed/(1-poss_transformed)
                    poss_transformed=np.log(poss_transformed)

                if 'arith' in a:
                    aggregations[aggrkey][qid]['aggr_forecasts'].append(np.sum(weights*poss_transformed)/np.sum(weights))
                    #aggregations[aggrkey][qid]['aggr_forecasts'].append(np.mean(poss_transformed))
                elif 'geom' in a:
                    aggregations[aggrkey][qid]['aggr_forecasts'].append(statistics.geometric_mean(poss_transformed))
                elif 'median' in a:
                    aggregations[aggrkey][qid]['aggr_forecasts'].append(np.median(poss_transformed))

                aggregations[aggrkey][qid]['options'].append(o)

            if 'odds' in a:
                odds=np.array(aggregations[aggrkey][qid]['aggr_forecasts'])
                aggregations[aggrkey][qid]['aggr_forecasts']=odds/(1+odds)
            elif 'logodds' in a:
                log_odds=np.array(aggregations[aggrkey][qid]['aggr_forecasts'])
                odds=np.exp(log_odds)
                aggregations[aggrkey][qid]['aggr_forecasts']=odds/(1+odds)

            if 'gjpextr' in a:
                p=np.array(aggregations[aggrkey][qid]['aggr_forecasts'])
                aggregations[aggrkey][qid]['aggr_forecasts']=((p**EXTR)/(((p**EXTR)+(1-p))**(1/EXTR)))
            elif 'postextr' in a:
                p=np.array(aggregations[aggrkey][qid]['aggr_forecasts'])
                aggregations[aggrkey][qid]['aggr_forecasts']=p**EXTR
            elif 'neyextr' in a:
                p=np.array(aggregations[aggrkey][qid]['aggr_forecasts'])
                d=n*(math.sqrt(3*n**2-3*n+1)-2)/(n**2-n-1)
                aggregations[aggrkey][qid]['aggr_forecasts']=p**d

            aggregations[aggrkey][qid]['options']=np.array(aggregations[aggrkey][qid]['options'])
            aggregations[aggrkey][qid]['aggr_forecasts']=np.array(aggregations[aggrkey][qid]['aggr_forecasts'])

            # Renormalize to 1
            Z=np.sum(aggregations[aggrkey][qid]['aggr_forecasts'])
            aggregations[aggrkey][qid]['aggr_forecasts']/=Z

    for a in aggregations.keys():
        briers=[]
        for qid in aggregations[a].keys():
            if len(aggregations[a][qid]['aggr_forecasts'])==0:
                continue
            aggregations[a][qid]['brier']=np.mean((aggregations[a][qid]['aggr_forecasts']-(aggregations[a][qid]['outcome']==aggregations[a][qid]['options']))**2)
            briers.append(aggregations[a][qid]['brier'])
        aggregations[a]['brier']=np.mean(np.array(briers))

    return aggregations

In [17]:
aggregations=get_aggregations(q_forecasts, questions)
team_aggregations=get_aggregations(team_forecasts, questions)
nonteam_aggregations=get_aggregations(nonteam_forecasts, questions)

Calculating the advantage of different types of groups.

In [18]:
l=[(aggregations[k]['brier'],k) for k in aggregations.keys()]
l.sort()
tl=[(team_aggregations[k]['brier'],k) for k in team_aggregations.keys()]
tl.sort()
ntl=[(nonteam_aggregations[k]['brier'],k) for k in nonteam_aggregations.keys()]
ntl.sort()

print('general: ')
for e in l:
    print(e[1], e[0])

print('')

print('team: ')
for e in tl:
    print(e[1], e[0])

print('')

print('nonteam: ')
for e in ntl:
    print(e[1], e[0])

print('')
print(individual_scores)

general: 
arith_probs_dec_befextr 0.1100069411444964
arith_probs_dec_neyextr 0.11131216703983897
arith_odds_dec_postextr 0.11226688140658732
arith_odds_dec_gjpextr 0.11434462795611515
arith_probs_dec_postextr 0.1161838039889886
arith_probs_dec_gjpextr 0.1163734307997773
arith_probs_dec_noextr 0.11919331910539008
arith_odds_dec_befextr 0.12110430495802299
geom_odds_nodec_noextr 0.12366579737648133
arith_probs_nodec_befextr 0.12386721226943251
geom_odds_nodec_neyextr 0.12477960246549023
arith_odds_nodec_postextr 0.12478666433777126
arith_probs_nodec_neyextr 0.12502557416508217
geom_probs_nodec_noextr 0.12534286737203398
arith_odds_dec_neyextr 0.12581824494525026
arith_odds_nodec_gjpextr 0.1267951132332894
geom_probs_nodec_neyextr 0.12681840586003373
arith_probs_nodec_postextr 0.13042996125447376
median_probs_nodec_noextr 0.1306004149415392
median_logodds_nodec_noextr 0.13060109909462075
median_odds_nodec_noextr 0.13060114217546281
arith_probs_nodec_gjpextr 0.13064769698757298
median_prob

We can see the following:

Forecasters outside of teams, without extremising, had a Brier score of 0.124, when looking at teams, the Brier score shrinks to 0.108, when then extremising with the GJP method, the Brier score shrinks again to 0.067.

* Nothing: Brier score is 0.124 (`arith_probs_nodec_noextr` in noteam)
* Decay: 0.115 (`arith_probs_dec_noextr` in noteam)
* Extremising: 0.0776 (`arith_probs_nodec_gjpextr` in noteam)
* All together: 0.0702 (`arith_probs_dec_gjpextr` in noteam)

Using teams provides an additional advantage:

* Team: 0.108 (`arith_probs_nodec_noextr` in team)
* All together: 0.062 (`arith_probs_dec_gjpextr` in team)

They also state that

> Note that we take the weighted mean first, and then transform; this works much better than transforming first and then averaging the transformed individual predictions. The transformation we used is: $\frac{p^a}{(p^a+(1-p))^{\frac{1}{a}}}$ with $a=3$.

This looks true: `arith_probs_dec_befextr` receives a Brier score of 0.082, while `arith_probs_dec_gjpextr` receives a Brier score of 0.070.

In [ ]:
GROUP_SCORE_SAMPLES=100
MAX_GROUP_SIZE=100

group_scores=dict()

for size in range(2, MAX_GROUP_SIZE+1):
    samples=[]
    for n in range(0, GROUP_SCORE_SAMPLES):
        sample_group=random.sample(users, size)
        score=group_score(user_forecasts, sample_group)
        samples.append(score)
    group_scores[size]=np.mean(np.array(samples))

for k in group_scores.keys():
    print(k, len((np.where(individual_scores>group_scores[k]))[0])/len(individual_scores))